# Comparing Reference Genomes

First, check you are in the correct directory.

In [ ]:
pwd

It should display something like:

`/home/manager/course_data/assembly/data`

So, we have done Pacbio and Illumina sequencing, and made a genome assembly - now what do we do? In many cases, researchers have previously performed
 assemblies of a model organism, but these assemblies may be imperfect due to various factors. Our goal here is to
 determine how our reference genome compares to a previous reference genome.

**Question:** Based on your work during the previous assembly module, can you think of a reason why assembly might not be perfect?

## Set Up

We have already placed a copy of the assembly that you performed yesterday into your current working directory:

Do the following command to ensure that it is present:

In [ ]:
ls PB.contigs.polished.fasta

We first need to convert the header of this file to something that is easier for our tools to read:

In [ ]:
{ echo ">tig00000001"; tail -n+2 PB.contigs.polished.fasta; } > PB.contigs.polished.reheader.fasta


You can then run the following command to make sure that you have performed the formatting correctly:

In [ ]:
head PB.contigs.polished.reheader.fasta

should return something similar to:

    >tig00000001
    TCTTATCTTCTTACTCTTATCTTCTTACTTTTCATTTCTTAGTCTTACTTTCTTCTTCTT
    ATCTTCTTACTGTTATCTTCTTACTTTTCATTCCTTACTCTTACTTACTTACTCTTATCT
    TCTTACTTTTCATTCCTTAGTCTTACTTACTTACTCTTACTTTCTTCTTCTTATCTTCTT
    ACTCTTATCTTCTTACTTTTCATTACTTAGTCTTACTTACTTACTCTTACTTACTTACTC
    TTATCTTCTTACTTTTCATTCCTTACTCTTACTTACTTACTCTTATCTTCTTACTTTTCA
    TTCCTTACTCTTACTTTCTTCTTCTTAGGTCCTTACTTTTAACTTCTTATTCTTACTTTC
    TTACTCTTACGTCCTTACTCTTACTTACTTACTCTTATCTTCTTACTTTTCATTCCTTAC
    TTTTCATTCCTTACTTTTCATTTCTTCATCTTATCTTCTTACTTTTCATTCCTTACTCTT
    ACTTACTTACTCTTATCTTCTTACTTTTCATTTCTTAATCATATATTCTTACTCATATAC

Now index this file so that we can use it for other parts of this practical:

In [ ]:
samtools faidx PB.contigs.polished.reheader.fasta

If these commands did not work, there is a copy of `PB.contigs.polished.reheader.fasta` in `annotation_backups/`.

## Visually Comparing Assemblies

Since we know from the Genome Assembly practical that our assembly was generated from sequencing chromosome 5 of a
_P. falciparum_ isolate, we can compare our results to a previously assembled version of the _P. falciparum_ genome. We
have pre-downloaded a version of the current _P. falciparum_ reference genome from [PlasmoDB](https://plasmodb.org) that only contains chromosome 5 and
placed it in your current directory. Run the following command to see what the first 10 lines of the file look like.

In [ ]:
head Pfalc_chr5_ref.fa

This should look similar to your own file that you assembled yesterday.

Next, we are going to use the tool `mummer` to align our assembly to the reference sequence of _P. falciparum_. Mummer
was developed to align entire bacterial genomes and is available on both [github](https://github.com/mummer4/mummer) and
through bioconda. We can align our assembly to the _P. falciparum_ reference with the following command:

In [ ]:
nucmer -p aln Pfalc_chr5_ref.fa PB.contigs.polished.reheader.fasta

This will generate the file "aln.delta". This file contains information on how the two reference genomes align, but is a
 difficult to interpret. Let's generate something that is a bit more human-readable:

In [ ]:
show-coords aln.delta > aln.coords

Let's take a look at the output of `show-coords` to see if we can learn anything. Run the following command:

In [ ]:
head aln.coords

You should see a table of information like this:

       [S1]     [E1] |   [S2]     [E2] | [LEN 1] [LEN 2] | [% IDY] | [TAGS]
    =====================================================================================
           1  236439 |       7  236291 |  236439  236285 |   99.87 | PfIT_05	tig00000001
      136356  136443 | 1031955 1031868 |      88      88 |   95.45 | PfIT_05	tig00000001
      136376  136448 |  423822  423750 |      73      73 |  100.00 | PfIT_05	tig00000001
      136376  136447 |  782656  782585 |      72      72 |  100.00 | PfIT_05	tig00000001
      136377  136445 |  862797  862729 |      69      69 |  100.00 | PfIT_05	tig00000001

The S1 and E1 columns represent the START and END coordinates in the sequence of the genome that you aligned to
(so the original Malaria reference genome) and S2 and E2 represent the START and END coordinates in the sequence
of your assembly (named tig00000001). LEN1 and LEN2 represent the length of the aligned segments and % IDY is how well
the two sequences match. So a % IDY of 100 means that the aligned segments match perfectly.

Now, lets use `mummerplot` to visualise this result to better understand what these alignments mean:

In [ ]:
mummerplot -l aln.delta --png

We can look at the resulting plot using the default image viewer on your virtual machine:

In [ ]:
eog out.png

This should give an image like the following:

![](images/MUMMER_1.png)

The y-axis is our pacbio assembly ordered from position 1 (the bottom) to position ~1,200,000 (the top). The reference
_P. falciparum_ genome is on the x-axis. Each place where the two sequences align perfectly is represented by a purple
line. You can see that this line is right in the middle of the plot, which means that base 10,000 of our assembly is the same
as base 10,000 of the reference, base 40,000 of our assembly is the same as base 40,000 in the reference, and so on. We
did a good job with our assembly compared to the reference genome!

The blue dots relate to parts of each genome that align multiple times. In other words, each blue dot is a "repeat" in
our assembly which matches the reference more than once.

**Questions**:

1. Is there an obvious issue with our assembly?

>_hint: look at the upper right corner of the plot_

2. Why do you think both ends of the reference genome align to the same part of our assembled genome?

>_hint: think about the structure of a chromosome and look at the lower right corner of the plot_

Here is another example of a mummerplot comparing assemblies between two different isolates of the bacteria H. pylori
from the mummer tutorial website:

![](images/MUMMER_2.png)

**Questions**:

1. What do you think the green segments represent in this image?
2. Why is the red line not centered in the plot and moves up or down?

Now you can move on to [Repetitive DNA](repetitive_dna.ipynb).